<a href="https://colab.research.google.com/github/olorunfemibabalola/Bias-Detection-NLP/blob/main/Inclusive_HR_Policy_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers gradio PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import torch
from transformers import pipeline
import PyPDF2

# 1. Initialize Llama 3.1 (using 4-bit for Colab efficiency)
model_id = "meta-llama/Llama-3.1-8B-Instruct"
generator = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True},
    device_map="auto",
)

def process_interaction(message, history, uploaded_file):
    # STEP 1: Detect Mode (Document Audit vs. Chat)
    if uploaded_file is not None:
        # --- MODE 2: DOCUMENT AUDITOR ---
        reader = PyPDF2.PdfReader(uploaded_file.name)
        text = " ".join([page.extract_text() for page in reader.pages])

        system_instr = "SYSTEM: You are a Senior HR Compliance Auditor. Analyze this document strictly for Gender, Race, and Age bias. List every biased sentence found. If none, state 'Document is Safe'."
        prompt = f"{system_instr}\n\nDOCUMENT CONTENT: {text[:3000]}\n\nAUDIT REPORT:"

    else:
        # --- MODE 1: SENTINEL CHATBOT ---
        system_instr = (
            "SYSTEM: You are an HR Policy Assistant. Answer the user's question clearly. "
            "IMPORTANT: If the user's input contains any gender, race, or age bias, "
            "you MUST point it out politely before answering."
        )
        prompt = f"{system_instr}\n\nUSER QUESTION: {message}\n\nASSISTANT RESPONSE:"

    # Generate output
    outputs = generator(prompt, max_new_tokens=400, do_sample=True, temperature=0.6)
    return outputs[0]["generated_text"].split("RESPONSE:")[-1].split("REPORT:")[-1].strip()

# 3. Professional Gradio UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🛡️ PolicyGuard: Bias-Aware Chatbot & Auditor")

    with gr.Row():
        with gr.Column(scale=3):
            chat_interface = gr.ChatInterface(fn=process_interaction, additional_inputs=[gr.File(label="Upload Policy for Audit")])
        with gr.Column(scale=1):
            gr.Markdown("### How to use\n1. Ask any HR policy question in the chat.\n2. To audit a full file, upload a PDF and type 'Please audit this' in the chat.")

demo.launch(debug=True)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct.
401 Client Error. (Request ID: Root=1-6962cf26-1e3e692720ac9fc60bd352f0;96fe0215-b245-4658-a29c-f7978e598892)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.1-8B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.